# MLP
多层感知器模型，采用三层网络，ReLU，最后softmax，进行MNist的softmax回归，代码和lr比较相近，只是我们需要大建的Model不一样而已，所以我们会意识到，其实很多的模型，在训练层面大致的代码都差不多，未来，我们会这些统一的东西，进行封装，目前，为了加深印象，我们依然，要自己手写，直到能够熟练地背诵下来。熟能生巧嘛！

In [3]:
from torchvision import datasets, transforms
import torch

 马上那个二货normalize又出来了，很恶心，我们先看看到底是个什么玩意儿

In [15]:
test_set = datasets.MNIST('./data', train = True, download = True, transform = transforms.Compose([ transforms.ToTensor() ]))
help(test_set)
help(torch.utils.data.dataset.Dataset)
print(type(test_set.train_data))
print(test_set.train_data.shape)
help(torch.ByteTensor)

Help on MNIST in module torchvision.datasets.mnist object:

class MNIST(torch.utils.data.dataset.Dataset)
 |  `MNIST <http://yann.lecun.com/exdb/mnist/>`_ Dataset.
 |  
 |  Args:
 |      root (string): Root directory of dataset where ``processed/training.pt``
 |          and  ``processed/test.pt`` exist.
 |      train (bool, optional): If True, creates dataset from ``training.pt``,
 |          otherwise from ``test.pt``.
 |      download (bool, optional): If true, downloads the dataset from the internet and
 |          puts it in root directory. If dataset is already downloaded, it is not
 |          downloaded again.
 |      transform (callable, optional): A function/transform that  takes in an PIL image
 |          and returns a transformed version. E.g, ``transforms.RandomCrop``
 |      target_transform (callable, optional): A function/transform that takes in the
 |          target and transforms it.
 |  
 |  Method resolution order:
 |      MNIST
 |      torch.utils.data.dataset.Da

In [17]:
print(test_set.train_data.numpy().mean(axis = (0, 1, 2))/255)
print(test_set.train_data.numpy().std(axis = (0, 1, 2))/255)

0.130660476274
0.308107803856


In [22]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]) #  为了让数据分布在-1-1之间啦

train_mnist_set = datasets.MNIST('./data', train = True, download = True,
                      transform = transform)
train_loader = torch.utils.data.DataLoader(train_mnist_set, batch_size = 32, shuffle = True)

test_mnist_set = datasets.MNIST('./data', train = False, download = True,
                      transform = transform)
test_loader = torch.utils.data.DataLoader(test_mnist_set, batch_size = 32, shuffle = True)


每次都废了个吃屎的劲儿才把loader准备好，不容易啊，自己的数据集的构造我们还没学呢，那个需要学会自己准备存储和读取数据

好，接着我们已经比较熟悉的，准备网络

In [24]:
import torch.nn as nn
import torch.nn.functional as F
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__() # 这里的傻逼用法，不是torch的问题，是python在多重继承时自己傻逼，然后让用户吃屎，才出现这么个恶心语法，不想研究的先记住吧
        self.fc1 = nn.Linear(784, 20, bias = True)
        self.fc2 = nn.Linear(20, 20, bias = True)
        self.fc3 = nn.Linear(20, 10, bias = True) # logits
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

OK，网络比较简单，一气呵成

In [28]:
help(enumerate)

Help on class enumerate in module builtins:

class enumerate(object)
 |  enumerate(iterable[, start]) -> iterator for index, value of iterable
 |  
 |  Return an enumerate object.  iterable must be another object that supports
 |  iteration.  The enumerate object yields pairs containing a count (from
 |  start, which defaults to zero) and a value yielded by the iterable argument.
 |  enumerate is useful for obtaining an indexed list:
 |      (0, seq[0]), (1, seq[1]), (2, seq[2]), ...
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.



In [52]:
import torch.optim as optim
from torch.autograd import Variable
net = MLPNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum = 0.5)

for epoch in range(100):
    for i, data in enumerate(train_loader, 0): # 0的意思是无论重跑几次，都从0开始迭代，避免多次试验时不稳定的问题
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad() # 等同于net.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    _, predicted = torch.max(outputs, 1) # 注意一定是dim = 1，0是batch
    acc = (predicted == labels).sum()
    total = labels.size(0)
    print("epoch: %d, Loss: %.2f, Accuracy: %.2f %%" % (epoch, loss.data[0], acc.data[0] * 100 / total))
        
        

epoch: 0, Loss: 0.23, Accuracy: 93.75 %
epoch: 1, Loss: 0.31, Accuracy: 87.50 %
epoch: 2, Loss: 0.14, Accuracy: 93.75 %
epoch: 3, Loss: 0.16, Accuracy: 90.62 %
epoch: 4, Loss: 0.52, Accuracy: 87.50 %
epoch: 5, Loss: 0.11, Accuracy: 96.88 %
epoch: 6, Loss: 0.04, Accuracy: 100.00 %
epoch: 7, Loss: 0.11, Accuracy: 93.75 %
epoch: 8, Loss: 0.18, Accuracy: 93.75 %
epoch: 9, Loss: 0.10, Accuracy: 93.75 %
epoch: 10, Loss: 0.11, Accuracy: 96.88 %
epoch: 11, Loss: 0.36, Accuracy: 87.50 %
epoch: 12, Loss: 0.04, Accuracy: 96.88 %
epoch: 13, Loss: 0.05, Accuracy: 93.75 %
epoch: 14, Loss: 0.05, Accuracy: 100.00 %
epoch: 15, Loss: 0.01, Accuracy: 100.00 %
epoch: 16, Loss: 0.06, Accuracy: 96.88 %
epoch: 17, Loss: 0.04, Accuracy: 100.00 %
epoch: 18, Loss: 0.02, Accuracy: 100.00 %
epoch: 19, Loss: 0.31, Accuracy: 90.62 %
epoch: 20, Loss: 0.14, Accuracy: 93.75 %
epoch: 21, Loss: 0.07, Accuracy: 96.88 %
epoch: 22, Loss: 0.01, Accuracy: 100.00 %
epoch: 23, Loss: 0.04, Accuracy: 100.00 %
epoch: 24, Loss: 0.

In [101]:
# 重新定义一下
test_loader = torch.utils.data.DataLoader(test_mnist_set, batch_size = None, shuffle = False) # None就是全部
for i, data in enumerate(test_loader, 0):
    inputs, labels = data
    inputs, labels = Variable(inputs), Variable(labels)
    outputs = net(inputs)
    _, predicted = torch.max(outputs, 1)
    
    total = labels.size(0)
    right = (predicted == labels).long().sum() # 只有使用long()才能避免傻逼torch的boolTensor累积溢出问题
    print("Accuracy: %.2f %%" % (right.data[0] * 100 / total))
    

Accuracy: 95.93 %


In [79]:
inputs, labels = next(iter(test_loader))

t = inputs.numpy()[0]
for i in range(t.shape[1]):
    for j in range(t.shape[2]):
        if t[0][i][j] > 0:
            print("1", end='')
        else:
            print("0", end='')
    print("")

inputs = Variable(torch.Tensor(t))
outputs = net(inputs[0])
print(outputs)
_, predicted = torch.max(outputs, 1)
print(predicted)

labels = labels[0:1]

print(labels[0])

print((predicted.data == labels).sum())


0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000001111110000000000000000
0000001111111111111111000000
0000001111111111111111000000
0000000000001011110111000000
0000000000000000001110000000
0000000000000000001110000000
0000000000000000011110000000
0000000000000000111100000000
0000000000000000111000000000
0000000000000000111000000000
0000000000000001110000000000
0000000000000011110000000000
0000000000000011100000000000
0000000000000111100000000000
0000000000001111000000000000
0000000000001110000000000000
0000000000011110000000000000
0000000000111110000000000000
0000000000111110000000000000
0000000000111000000000000000
0000000000000000000000000000
Variable containing:

Columns 0 to 7 
  7.4015   0.6492   9.5151  15.3454 -22.6956   4.3543 -35.3334  28.3885

Columns 8 to 9 
  4.3321  13.3023
[torch.FloatTensor of size 1x10]

Variable 